In [6]:
import numpy as np
import matplotlib.pyplot as plt
import alpha_recoil_sim as ar
import importlib, glob, os, shutil, subprocess, pickle

In [8]:
## first make a list of all the daughter isotopes we need
path = "/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/Sphere_Recoils_MC"

iso_list = ['Ac-225', 'Pb-212', 'At-211', 'Th-227', 'Ra-223', 'Po-216', 'He-4']

iso_dict = {}

for iso in iso_list:
    ciso = iso[:2].lower()
    cA = iso.split('-')[1]
    iso_dict[iso] = ar.parse_decay_chain(path + "/decay_data/" + ciso + "_" + cA + "_decay_chain.txt")

In [9]:
## print latex table of major alphas for each isotope for the note
alpha_mass=4
for iso in iso_list:

    full_list = iso_dict[iso].keys()
    dlist = []
    for k in full_list:
        if 'decays' in k:
            dlist.append(k[:6])

    curr_nr_list = []
    curr_nr_eng = []
    curr_alpha_eng = []
    curr_parents = []

    for k in dlist:

        decay = iso_dict[iso][k + "_decays"]
        daught = iso_dict[iso][k + "_daughters"]

        for j in range(len(daught)):
            
            if daught[j] not in curr_nr_list:
                daughter_mass = float(daught[j].split("-")[-1])
                decay_NR_energy = decay[j,1] * alpha_mass/daughter_mass
                if(decay_NR_energy > 0):
                    curr_nr_list.append(daught[j])
                    curr_nr_eng.append(decay_NR_energy)
                    curr_alpha_eng.append(decay[j,1])
                    curr_parents.append(k)

    #print(iso, curr_nr_eng, curr_nr_list)
    iso_mass = float(iso.split("-")[-1])
    iso_symb = iso.split("-")[0]
    s1 = "$^{%d}$%s & "%(iso_mass, iso_symb)
    for j in range(len(curr_alpha_eng)):
        cm = float(curr_parents[j].split("-")[-1])
        cs = curr_parents[j].split("-")[0]
        s1 += "$^{%d}$%s (%.1f), " % (cm, cs, curr_alpha_eng[j]/1000) 

    s1 = s1[:-2] #drop comma
    s1 += " & "

    for j in range(len(curr_nr_eng)):
        cm = float(curr_nr_list[j].split("-")[-1])
        cs = curr_nr_list[j].split("-")[0]
        s1 += "$^{%d}$%s (%d), " % (cm, cs, curr_nr_eng[j]) 

    s1 = s1[:-2] #drop comma
    s1 += " \\\\ "

    print(s1)


$^{225}$Ac & $^{225}$Ac (5.8), $^{221}$Fr (6.3), $^{217}$At (7.1), $^{213}$Bi (5.9), $^{213}$Po (8.4) & $^{221}$Fr (105), $^{217}$At (116), $^{213}$Bi (132), $^{209}$Tl (112), $^{209}$Pb (160) \\ 
$^{212}$Pb & $^{212}$Bi (6.1), $^{212}$Po (8.8) & $^{208}$Tl (117), $^{208}$Pb (168) \\ 
$^{211}$At & $^{211}$At (5.9), $^{211}$Po (7.5) & $^{207}$Bi (113), $^{207}$Pb (143) \\ 
$^{227}$Th & $^{227}$Th (6.0), $^{223}$Ra (5.9), $^{219}$Rn (6.8), $^{215}$Po (7.4), $^{211}$Bi (6.6), $^{211}$Po (7.5) & $^{223}$Ra (108), $^{219}$Rn (107), $^{215}$Po (126), $^{211}$Pb (140), $^{207}$Tl (127), $^{207}$Pb (143) \\ 
$^{223}$Ra & $^{223}$Ra (5.9), $^{219}$Rn (6.8), $^{215}$Po (7.4), $^{211}$Bi (6.6), $^{211}$Po (7.5) & $^{219}$Rn (107), $^{215}$Po (126), $^{211}$Pb (140), $^{207}$Tl (127), $^{207}$Pb (143) \\ 
$^{216}$Po & $^{216}$Po (6.8), $^{212}$Bi (6.1), $^{212}$Po (8.8) & $^{212}$Pb (127), $^{208}$Tl (117), $^{208}$Pb (168) \\ 
$^{4}$He   \\ 


In [10]:
## go through the dictionary and make a list of all alpha decay daughters we need SRIM simulations for
daughter_list = []
for iso in iso_list:
    if(iso == "He-4"):
        daughter_list.append(iso)
        continue
    
    curr_dict = iso_dict[iso]
    
    curr_keys = curr_dict.keys()

    for k in curr_keys:
        if not '_daughters' in k: continue

        curr_daught = curr_dict[k]
        curr_alpha = curr_dict[k[:6] + "_decays"][:,1] > 0

        for j in range(len(curr_daught)):
            if( curr_alpha[j] and not curr_daught[j] in daughter_list):
                daughter_list.append(curr_daught[j]) 

print(daughter_list)


['Fr-221', 'At-217', 'Bi-213', 'Tl-209', 'Pb-209', 'Tl-208', 'Pb-208', 'Bi-207', 'Pb-207', 'Ra-223', 'Rn-219', 'Po-215', 'Pb-211', 'Tl-207', 'Pb-212', 'He-4']


In [30]:
## now generate the TRIM input files from the templates for the list of isotopes above
importlib.reload(ar)

material_list = ["SiO2", "Au", "Ag", "tissue"]
recoil_energy = {"NR": 200, "alpha": 10000} ## keV
energy_step_to_save = {"NR": 1000, "alpha": 50000} ## eV, step to save in EXYZ file
slab_thickness = {"NR": {"SiO2": 2000, "Au": 1000, "Ag": 1000, "tissue": 3000}, ## angstroms, nucleus
                  "alpha": {"SiO2": 2000000, "Au": 400000, "Ag": 600000, "tissue": 3000000}} ## angstroms, alpha
num_events = 10000 ## number of events for each isotope and material

fidx = 0
for mat in material_list:

    template_file = path + "/TRIM_input_files/TRIM.IN_%s.txt"%mat

    with open(template_file, 'r') as tf:
        template_lines = tf.readlines()

    for iso in daughter_list:

        if(iso == "He-4"):
            tag = "alpha"
        else:
            tag = "NR"

        output_file = path + "/TRIM_input_files/TRIM.IN_%d"%fidx
        print("Writing %d: %s %s"%(fidx, iso, mat))

        with open(output_file, 'wt') as of:
            
            for lidx, l in enumerate(template_lines):

                if lidx == 0: ## write the first line as a header unchanged
                    of.write(template_lines[0])
                    continue

                ## update ion information
                if template_lines[lidx - 1].startswith("Ion:"):
                    Z, A = ar.get_Z_A_for_iso(iso)
                    curr_recoil_energy = recoil_energy[tag]
                    ion_line = "    %d    %d    %d    0    %d    1    %d\n"%(Z, A, curr_recoil_energy, num_events, num_events)
                    of.write(ion_line)
                elif template_lines[lidx - 1].startswith("Target material"):
                    new_start = "%s (%d) into "%(iso, recoil_energy[tag])
                    newl = l[0] + new_start + l[14:]
                    of.write(newl)
                elif template_lines[lidx - 1].startswith("Diskfiles"):
                    save_line = "                          0       0           0       0               0                               %d\n"%energy_step_to_save[tag]
                    of.write(save_line)
                elif template_lines[lidx - 2].startswith("Layer"):
                    lineparts = l.split()
                    layer_num = int(lineparts[0])
                    layer_name = ""
                    last_idx = 1
                    for part in lineparts[1:]:
                        layer_name += part + " "
                        last_idx += 1
                        if part.endswith('"'):
                            break
                    layer_line = " %d      %s           %d "%(layer_num, layer_name, slab_thickness[tag][mat])
                    for part in lineparts[last_idx+1:]:
                        layer_line += part + " "
                    of.write(layer_line + "\n")
                else:
                    of.write(l)

        fidx += 1


Writing 0: Fr-221 SiO2
Writing 1: At-217 SiO2
Writing 2: Bi-213 SiO2
Writing 3: Tl-209 SiO2
Writing 4: Pb-209 SiO2
Writing 5: Tl-208 SiO2
Writing 6: Pb-208 SiO2
Writing 7: Bi-207 SiO2
Writing 8: Pb-207 SiO2
Writing 9: Ra-223 SiO2
Writing 10: Rn-219 SiO2
Writing 11: Po-215 SiO2
Writing 12: Pb-211 SiO2
Writing 13: Tl-207 SiO2
Writing 14: Pb-212 SiO2
Writing 15: He-4 SiO2
Writing 16: Fr-221 Au
Writing 17: At-217 Au
Writing 18: Bi-213 Au
Writing 19: Tl-209 Au
Writing 20: Pb-209 Au
Writing 21: Tl-208 Au
Writing 22: Pb-208 Au
Writing 23: Bi-207 Au
Writing 24: Pb-207 Au
Writing 25: Ra-223 Au
Writing 26: Rn-219 Au
Writing 27: Po-215 Au
Writing 28: Pb-211 Au
Writing 29: Tl-207 Au
Writing 30: Pb-212 Au
Writing 31: He-4 Au
Writing 32: Fr-221 Ag
Writing 33: At-217 Ag
Writing 34: Bi-213 Ag
Writing 35: Tl-209 Ag
Writing 36: Pb-209 Ag
Writing 37: Tl-208 Ag
Writing 38: Pb-208 Ag
Writing 39: Bi-207 Ag
Writing 40: Pb-207 Ag
Writing 41: Ra-223 Ag
Writing 42: Rn-219 Ag
Writing 43: Po-215 Ag
Writing 44: Pb

In [31]:
## code to run on windows computer to run TRIM jobs
input_files = glob.glob("C:/SRIM/Sphere_Recoils_MC/TRIM_input_files/*")

replace_files = False

destination_file = r"C:/SRIM/TRIM.IN"
trim_path = r"C:\SRIM"
output_dir = r"C:\SRIM\output_files_batch"
exyz_file = r"C:\SRIM\SRIM Outputs\EXYZ.txt"

for file in input_files:
    if file.endswith('.txt'): continue

    file_idx = file.split("_")[-1]

    if(os.path.isfile(output_dir + "\exyz_%s.txt"%file_idx) and not replace_files):
        print(output_dir + "\exyz_%s.txt"%file_idx + " already exists, skipping")
        continue

    shutil.copy(file, destination_file)
    print("Copied %s to %s"%(file, destination_file)) 

    print(trim_path)
    os.chdir(trim_path)
    subprocess.call("./TRIM.exe")

    shutil.copy(exyz_file, output_dir + "\exyz_%s.txt"%file_idx)


In [32]:
## parse exyz.txt files from TRIM and save in more efficient python data structure

data_path = '/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/SRIM_Data/'

data_dict = {}

A_to_nm = 0.1 #convert angstrom to nm

fidx = 0
for mat in material_list:
    for iso in daughter_list:

        with open(data_path + "exyz_%d.txt"%fidx) as df:
            lines = df.readlines()
        
        ### fix order for Pb-212
        #if(iso == "Pb-212"): continue

        event_dict = {}
        curr_event = -1
        curr_traj_data = []
        for l in lines:
            if not l.startswith("0"): continue 

            curr_dat = l.strip().split()
            line_event = int(curr_dat[0])
            if(curr_event == -1):
                curr_event = line_event

            ## fix rare issue with SRIM files
            try:
                test = [float(curr_dat[1]), float(curr_dat[2]), float(curr_dat[3]), float(curr_dat[4])]
            except:
                continue

            if( abs(line_event - curr_event) < 0.1):
                curr_traj_data.append([float(curr_dat[1]), float(curr_dat[2])*A_to_nm, float(curr_dat[3])*A_to_nm, float(curr_dat[4])*A_to_nm])
            else:
                if( line_event % 10000 == 0 ): 
                    print("Working on %s in %s, event %d"%(iso, mat, line_event))
                event_dict[curr_event] = np.array(curr_traj_data, dtype=float)
                curr_event = line_event
                curr_traj_data = [[float(curr_dat[1]), float(curr_dat[2])*A_to_nm, float(curr_dat[3])*A_to_nm, float(curr_dat[4])*A_to_nm],]

        data_dict[iso + "_" + mat] = event_dict


        fidx += 1

with open(data_path + 'SRIM_MC_events.pkl', 'wb') as f:
    pickle.dump(data_dict, f)
    

Working on Fr-221 in SiO2, event 10000
Working on At-217 in SiO2, event 10000
Working on Bi-213 in SiO2, event 10000
Working on Tl-209 in SiO2, event 10000
Working on Pb-209 in SiO2, event 10000
Working on Tl-208 in SiO2, event 10000
Working on Pb-208 in SiO2, event 10000
Working on Bi-207 in SiO2, event 10000
Working on Pb-207 in SiO2, event 10000
Working on Ra-223 in SiO2, event 10000
Working on Rn-219 in SiO2, event 10000
Working on Po-215 in SiO2, event 10000
Working on Pb-211 in SiO2, event 10000
Working on Tl-207 in SiO2, event 10000
Working on He-4 in SiO2, event 10000
Working on Fr-221 in Au, event 10000
Working on At-217 in Au, event 10000
Working on Bi-213 in Au, event 10000
Working on Tl-209 in Au, event 10000
Working on Pb-209 in Au, event 10000
Working on Tl-208 in Au, event 10000
Working on Pb-208 in Au, event 10000
Working on Bi-207 in Au, event 10000
Working on Pb-207 in Au, event 10000
Working on Ra-223 in Au, event 10000
Working on Rn-219 in Au, event 10000
Working on

In [38]:
data_dict['He-4_SiO2'][9901]

array([[ 2.0000e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       [ 1.9885e+02,  5.1044e-01, -4.5704e-03, -1.0440e-03],
       [ 1.9554e+02,  1.0136e+00, -1.3814e-02, -3.0518e-03],
       [ 1.9498e+02,  1.5179e+00, -3.2717e-02,  7.9314e-03],
       [ 1.9388e+02,  2.5253e+00, -7.8275e-02,  2.7385e-02],
       [ 1.7758e+02,  5.5378e+00, -2.4865e-01,  1.6133e-01],
       [ 1.7609e+02,  6.0269e+00, -2.7005e-01,  2.1465e-01],
       [ 1.7563e+02,  6.5143e+00, -2.8308e-01,  2.7673e-01],
       [ 1.7465e+02,  7.9757e+00, -3.2050e-01,  4.5800e-01],
       [ 1.7362e+02,  9.4345e+00, -3.5910e-01,  6.3885e-01],
       [ 1.7267e+02,  1.0405e+01, -3.8038e-01,  7.6270e-01],
       [ 1.7168e+02,  1.1374e+01, -4.0222e-01,  8.9228e-01],
       [ 1.6802e+02,  1.1858e+01, -4.0906e-01,  9.5758e-01],
       [ 1.6411e+02,  1.2339e+01, -3.9832e-01,  1.0254e+00],
       [ 1.6368e+02,  1.2817e+01, -3.6996e-01,  1.0845e+00],
       [ 1.5006e+02,  1.3773e+01, -3.1601e-01,  1.2041e+00],
       [ 1.4962e+02,  1.

In [ ]:
#### now do simulations for the alphas themselves